# 1. Import Libraries and Data

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import time
import datetime
import sklearn
import statsmodels
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.neural_network
import sklearn.ensemble
import sklearn.gaussian_process
import tensorflow as tf
from scipy import stats
# Check SUMO_HOME is set
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("Please declare environment variable 'SUMO_HOME'")
    
import traci
import sumolib
import altair as alt
import folium
import matplotlib.pyplot as plt
import csv

In [2]:
col_types = {
    'count_point_id': 'string',
    'direction_of_travel': 'string',
    'count_date': 'string',
    'hour': 'string',
    'road_name': 'string',
    'road_type': 'string',
    'latitude': 'float',
    'longitude': 'float',
    'link_length_km': 'float',
    'pedal_cycles': 'int',
    'two_wheeled_motor_vehicles': 'int',
    'cars_and_taxis': 'int',
    'buses_and_coaches': 'int',
    'lgvs': 'int',
    'all_hgvs': 'int',
    'all_motor_vehicles': 'int' 
}

cols = list(col_types.keys())

dft_counts = pd.read_csv('dft_count_swansea.csv', sep=',', header=0,
                         index_col=None, dtype=col_types, usecols=cols, na_values='')
dft_counts['count_date'] = pd.to_datetime(dft_counts['count_date'], format= '%Y-%m-%d')
dft_counts

,count_point_id,direction_of_travel,count_date,hour,road_name,road_type,latitude,longitude,link_length_km,pedal_cycles,two_wheeled_motor_vehicles,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,all_motor_vehicles
0,931798,E,2003-03-20,7,U,Minor,51.613028,-4.004437,NaN,0,0,63,0,7,1,71
1,931798,E,2003-03-20,8,U,Minor,51.613028,-4.004437,NaN,0,0,290,4,12,1,307
2,931798,E,2003-03-20,9,U,Minor,51.613028,-4.004437,NaN,1,0,115,1,14,1,131
3,931798,E,2003-03-20,10,U,Minor,51.613028,-4.004437,NaN,0,1,99,1,10,2,113
4,931798,E,2003-03-20,11,U,Minor,51.613028,-4.004437,NaN,0,1,98,1,8,1,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16219,931802,N,2003-06-18,8,U,Minor,51.673471,-3.928426,NaN,0,1,219,6,13,6,245
16220,931802,N,2003-06-18,9,U,Minor,51.673471,-3.928426,NaN,0,0,101,2,22,4,129
16221,931802,N,2003-06-18,10,U,Minor,51.673471,-3.928426,NaN,0,0,101,2,20,4,127
16222,931802,N,2003-06-18,11,U,Minor,51.673471,-3.928426,NaN,0,0,99,6,12,16,133


# 2. Data Analysis and Preparation

In [3]:
def unstack(df):
    # unstack hourly counts
    unstacked_df = df.set_index(['count_point_id','direction_of_travel','count_date','hour'])
    unstacked_df = unstacked_df[['all_motor_vehicles']]
    unstacked_df = unstacked_df.unstack(level=-1)
    unstacked_df.reset_index(inplace=True)
    
    # flatten multi-level col index and rename
    unstacked_df.columns = unstacked_df.columns.to_flat_index()
    col_names = [a for a in unstacked_df.columns]
    col_names = [name[0] if col_names.index(name) <= 2 else name[1] for name in col_names]
    unstacked_df.columns = col_names
    ordered_col_names = ['count_point_id','direction_of_travel','count_date',
                         '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']
    unstacked_df = unstacked_df[ordered_col_names]
    return unstacked_df

unstacked_counts = unstack(dft_counts)

In [4]:
def get_cp_date_ranges(df):
    cps = [cp_id for cp_id in pd.unique(df['count_point_id'])]
    min_date = []
    max_date = []
    for cp_id in cps:
        min_date.append(df.loc[df['count_point_id'] == cp_id]['count_date'].min())
        max_date.append(df.loc[df['count_point_id'] == cp_id]['count_date'].max())
    cp_date_ranges = pd.DataFrame(
        {
            'cp_id': cps,
            'min_date': min_date,
            'max_date': max_date
        })
    return cp_date_ranges


def get_cp_dates(df):
    cps = [cp_id for cp_id in pd.unique(df['count_point_id'])]
    cps_longform = []
    dates = []
    for cp_id in cps:
        filtered_df = df.loc[df['count_point_id'] == cp_id]
        cps_longform = cps_longform + [i for i in df.loc[df['count_point_id'] == cp_id]['count_point_id']]
        dates = dates + [i for i in df.loc[df['count_point_id'] == cp_id]['count_date']]
    cp_dates = pd.DataFrame(
        {
            'cp_id': cps_longform,
            'dates': dates
        })
    return cp_dates

cp_date_ranges = get_cp_date_ranges(unstacked_counts)
cp_dates = get_cp_dates(unstacked_counts)

# Visualise
selector = alt.selection_interval(encodings=['y'])

dates_range = alt.Chart(cp_date_ranges).mark_bar().encode(
    x=alt.X('cp_id:O', title='Count Point ID'),
    y=alt.Y('min_date', title='Minimum - Maximum Date'),
    y2=alt.Y2('max_date', title=None),
    tooltip=[ 'cp_id','min_date', 'max_date']
).properties(
    title = 'Count Point Active Dates',
    width=600,
    height=400
)

dates = alt.Chart(cp_dates).mark_circle(color='orange').encode(
    x=alt.X('cp_id:O'),
    y=alt.Y('dates'),
    tooltip=['cp_id','dates']
).properties(
    width=600,
    height=400
).add_selection(
    selector
)

dates_range + dates

alt.LayerChart(...)

In [5]:
# group counts by cp id, year, and direction of travel and compute the mean for each group
# this actually doesnt reduce the number of rows at all
grouped_counts = unstacked_counts.groupby(['count_point_id',
                                           unstacked_counts['count_date'].dt.year,
                                           'direction_of_travel']).mean().reset_index()

# copy grouped counts and drop non count data
counts_for_model = grouped_counts.drop(columns=['count_point_id','count_date','direction_of_travel'])
counts_for_model

,7,8,9,10,11,12,13,14,15,16,17,18
0,346.0,624.0,280.0,234.0,246.0,265.0,258.0,269.0,299.0,302.0,364.0,222.0
1,235.0,445.0,272.0,268.0,319.0,349.0,377.0,343.0,474.0,567.0,575.0,288.0
2,308.0,561.0,280.0,216.0,280.0,247.0,291.0,263.0,245.0,303.0,345.0,236.0
3,306.0,539.0,313.0,281.0,300.0,335.0,341.0,386.0,511.0,625.0,603.0,376.0
4,301.0,368.0,196.0,185.0,168.0,169.0,182.0,182.0,222.0,235.0,213.0,139.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1347,459.0,771.0,557.0,532.0,612.0,662.0,626.0,747.0,969.0,1203.0,1236.0,890.0
1348,1174.0,1004.0,747.0,629.0,533.0,641.0,635.0,662.0,686.0,801.0,802.0,521.0
1349,519.0,791.0,536.0,590.0,572.0,691.0,679.0,803.0,943.0,1240.0,1299.0,936.0
1350,1226.0,1104.0,750.0,692.0,641.0,765.0,674.0,636.0,690.0,967.0,837.0,614.0


In [6]:
def count_outliers(data):
    '''
    Function to count the number of outliers (according to the scipy z-score)
    found within the traffic count data.
    
    '''
    outlier_count = {}
    for col in data.columns:
        bools = (np.abs(stats.zscore(data[col].astype(float))) < 3)
        if col not in outlier_count:
            outlier_count[col] = len(data)-len(data[bools])
        
    return outlier_count
   
count_outliers(counts_for_model)

{'7': 49,
 '8': 40,
 '9': 40,
 '10': 37,
 '11': 27,
 '12': 27,
 '13': 28,
 '14': 37,
 '15': 36,
 '16': 45,
 '17': 35,
 '18': 32}

Although it appears there are several outliers present across all CPs for each hour, as shown below, these are not truly outliers. The abnormally large values are just from high traffic flow sections of the network, like the M4. If we were to remove the outliers, this would remove 4 specific count points, thus loosing detail about those specific locations.

In [7]:
bools = (np.abs(stats.zscore(counts_for_model.astype(float))) < 3).all(axis=1)
idx = np.where(bools==False)
print(pd.unique(grouped_counts.iloc[idx]['count_point_id']))
grouped_counts.iloc[idx].loc[grouped_counts['count_point_id']=='40504'].head()

['20504' '30505' '40504' '503']


,count_point_id,count_date,direction_of_travel,7,8,9,10,11,12,13,14,15,16,17,18
145,40504,2000,W,1825.0,2587.0,1727.0,1594.0,1379.0,1445.0,1485.0,1523.0,1744.0,2387.0,2739.0,2118.0
147,40504,2001,W,1882.0,2089.0,2094.0,1539.0,1384.0,1416.0,1486.0,1528.0,1643.0,2377.0,2553.0,1914.0
149,40504,2002,W,2229.0,2801.0,1937.0,1790.0,1691.0,1668.0,1658.0,1728.0,2102.0,2591.0,2824.0,2007.0
150,40504,2005,E,2605.0,2673.0,1763.0,1503.0,1656.0,1699.0,1715.0,1999.0,2306.0,2631.0,2439.0,1566.0
151,40504,2005,W,2241.0,3058.0,2021.0,1621.0,1587.0,1609.0,1610.0,1740.0,1905.0,2454.0,2647.0,2170.0


In [8]:
# fix numpy seed
SEED = 202
np.random.seed(SEED) 

# split data into train and test sets
x = np.array(counts_for_model.drop('18', axis=1))
y = np.array(counts_for_model['18'])
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    x, y, test_size=0.4, random_state=22)

In [9]:
def get_scaler(scaler_type='robust'):
    '''
    Create and return sklearn scaler object that is fitted to the x_trian data. 
    Various scalers can be used, just pass a different string to scaler_type arg. 
    
    '''
    if scaler_type == 'standard':
        scaler = sklearn.preprocessing.StandardScaler()
    elif scaler_type == 'minmax':
        scaler = sklearn.preprocessing.MinMaxScaler()
    elif scaler_type == 'robust':
        scaler = sklearn.preprocessing.RobustScaler()
        
    # fit scaler to train data
    scaler.fit(x_train)
    return scaler


def scale_data(scaler, data_to_be_scaled):
    '''Function to scale data using the chosen scaler.'''
    scaled_data = scaler.transform(data_to_be_scaled)
    return scaled_data


# create scaler
scaler = get_scaler(scaler_type='robust')

# scale data
scaled_x_train = scale_data(scaler, x_train)
scaled_x_test = scale_data(scaler, x_test)

# 3. Traffic Prediction
In this section various models are trained to predict the 11th hour traffic counts given the previous 10.

## 3.1 Linear Regression
Baseline linear regression model.

We try creating an autoregression model. See [this machine learning mastery tutorial](https://machinelearningmastery.com/autoregression-models-time-series-forecasting-python/) for more details. 

> Because the regression model uses data from the same input variable at previous time steps, it is referred to as an autoregression (regression of self).

Alternativly, the [HCrystal Ball package](https://hcrystalball.readthedocs.io/en/stable/examples/tutorial/wrappers/02_ar_modelling_in_sklearn.html) could be used to wrap any sklearn regression model as an autoregressive model.

Some good papers that talk about previous models being used:
- [Gaussian Process Regression Based Traffic Modeling and Prediction in High-Speed Networks](https://ieeexplore.ieee.org/abstract/document/7841857)
- [Spatiotemporal Traffic Prediction Using Hierarchical Bayesian Modeling](https://www.mdpi.com/1999-5903/13/9/225/htm)
- [This one](http://thomas-liebig.eu/wordpress/wp-content/papercite-data/pdf/schnitzler14.pdf)
- [A bayesian network approach to traffic flow forecasting](https://ieeexplore.ieee.org/abstract/document/1603558?casa_token=EaS_xbcxIzEAAAAA:S4JezzP0kpRUWmj8ura7R1Pn89vXLN6Z-LPeJ1MmDOcbGymxb-q_WQCqg8jFZUc4IOD4_eb4g8g)
- Ridge regression model directly from sklearn.

### Grid Search CV
For attributes of the grid search results, look at the [Grid Search CV page](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=grid%20search%20cv#sklearn.model_selection.GridSearchCV) on the sklearn website. Similarly, for an explanation of CV, look at the [releveant sklearn user guide page](https://scikit-learn.org/stable/modules/cross_validation.html#). For an explanation of the metrics found in the `gscv` object, please refer to the 'returns' section of [this page](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate).

[This answer](https://stats.stackexchange.com/a/378464) cleared it up beautifully! 

[This documentation on storemagic](https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html) can be used to store and load grid search CV variables to save on computation time.

In [10]:
def optimise(model, params, x_train=scaled_x_train, y_train=y_train):
    '''
    Function to use Grid Search Cross Validation (GSCV) to optimise the hyperparameters
    of the sklearn models. The GSCV object is returned.
    
    '''
    start = time.time()
    grid_search = sklearn.model_selection.GridSearchCV(model, params, n_jobs=5, cv=5,
                                                      return_train_score=True, verbose=4)
    grid_search.fit(x_train, y_train)
    print(f'Found optimal hyperparameter combination for {model} in {np.round(time.time()-start,4)} seconds')
    return grid_search

In [11]:
# model
lr = sklearn.linear_model.LinearRegression()

# params
lr_params = {
    'fit_intercept': [True, False],
    'normalize': [True, False],
    'positive': [True, False]
}

# optimse
lr_gscv = optimise(lr, lr_params)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Found optimal hyperparameter combination for LinearRegression() in 1.34 seconds


In [12]:
def get_sklearn_ypred(model_gscv):
    '''
    Function to get predicted values from the sklearn models.
    This is done by extracting the best estimator from the gridsearchCV
    objects, and predicting the y values from the x_test set.
    
    '''
    best_estimator_idx = model_gscv.best_index_
    y_pred = model_gscv.best_estimator_.predict(scaled_x_test)
    
    return y_pred


In [13]:
lr_gscv.best_params_

{'fit_intercept': True, 'normalize': True, 'positive': False}

In [14]:
# get y predictions
lr_y_pred = get_sklearn_ypred(lr_gscv)

## 3.2 Random Forest
The next model to be used is a Random Forest. More information about it can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor).

```python
# model to be optimised
rf = sklearn.ensemble.RandomForestRegressor()

# parameters to search in dicts
rf_params = {
    'n_estimators': [100, 200, 500, 1000],
    'criterion': ['mse','mae'],
    'max_depth': [None, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}

rf_gscv = optimise(rf, rf_params) # gscv stands for grid search cross validation
# store gscv object to access across kernel sessions
%store rf_gscv

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Found optimal hyperparameter combination for RandomForestRegressor() in 220.164 seconds
Stored 'rf_gscv' (GridSearchCV)
```

In [15]:
# load gscv object from store as it takes a while otherwise. 
%store -r rf_gscv
rf_gscv

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=5,
             param_grid={'criterion': ['mse', 'mae'],
                         'max_depth': [None, 5, 10],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 500, 1000]},
             return_train_score=True, verbose=4)

In [16]:
rf_gscv.best_params_

{'criterion': 'mae',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 100}

In [17]:
rf_y_pred = get_sklearn_ypred(rf_gscv)

## 3.2 (Alt) Neural Network
In this section we shall try a neural network model from sklearn.

```python
# mlp model
mlp = sklearn.neural_network.MLPRegressor()

# params
mlp_params = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.00001, 0.0001, 0.001],
    'learning_rate': ['constant',' invscaling', 'adaptive'],
    'max_iter': [100, 250, 500, 1000]
}
    
mlp_gscv = optimise(mlp, mlp_params)
%store mlp_gscv

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Found optimal hyperparameter combination for MLPRegressor() in 298.5623 seconds
Stored 'mlp_gscv' (GridSearchCV)


# get gscv object from store
%store -r mlp_gscv
mlp_gscv

mlp_y_pred = get_sklearn_ypred(mlp_gscv)
mlp_gscv.best_score_
```

## 3.2 (Alt) Gaussian Process
Here we use a Gaussian Process Regression (GPR) model. More information about it can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor).

```python
# model
gp = sklearn.gaussian_process.GaussianProcessRegressor()

# params
gp_params = {
    #'kernel': [sklearn.gaussian_process.kernels.DotProduct(), 
    #           sklearn.gaussian_process.kernels.ConstantKernel()],
    'normalize_y': [False, True]
}

gp_gscv = optimise(gp, gp_params)

gp_y_pred = get_sklearn_ypred(gp_gscv)
```

## 3.3 CNN-LSTM
In this section we use a Convolutional Nerual Network with Long-Short-Term-Memory layers to predict the traffic flow. This is implemented using Tensorflow. 

To use sklearn's Grid Search Cross Validation, the model needs to wrapped. More information can be found [here](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/).
- [Cross-validation: evaluating estimator performance](https://scikit-learn.org/stable/modules/cross_validation.html#) 

First of all, the data needs to be scaled - feature wise. It is not advised to scale the row of data as then you lose the realtionships between the features for each row of data as they will be different. There are various scalers out there, [this page](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py) compares all the scalers sklearn offer - may be a good starting point for the discussion. 

In [18]:
def reshape_data(x_data, verbose=False):
    '''
    Function to reshape data into a format compatable with Conv1DLSTM layers.
    Expected shape = (samples, time, rows, channels)
    This reshaping is necessary for all input (x) data. 
    
    '''
    if verbose:
        print(f'Original data shape: {x_data.shape}')
        print(f'One sample shape: {x_data.shape[0]}')
        print(f'An example sample: {x_data[0]}')
    
    sample_size = x_data.shape[0] # number of samples in x data
    time_steps  = x_data.shape[1] # number of features in x data
    input_dimension = 1 # each feature is represented by 1 number
    
    reshaped_x_data = x_data.reshape(sample_size,time_steps,input_dimension)
    
    if verbose:
        print(f'Reshaped data shape: {reshaped_x_data.shape}')
        print(f'One sample shape: {reshaped_x_data.shape[0]}')
        print(f'An example sample: {reshaped_x_data[0]}')    
    
    return reshaped_x_data

# reshape scaled x data
reshaped_x_train = reshape_data(scaled_x_train, verbose=True)
reshaped_x_test = reshape_data(scaled_x_test, verbose=False)

Original data shape: (811, 11)
One sample shape: 811
An example sample: [-0.25396825 -0.22074689 -0.27627628 -0.26613704 -0.25223614 -0.22876367
 -0.24625624 -0.24523612 -0.28678118 -0.25771076 -0.30837004]
Reshaped data shape: (811, 11, 1)
One sample shape: 811
An example sample: [[-0.25396825]
 [-0.22074689]
 [-0.27627628]
 [-0.26613704]
 [-0.25223614]
 [-0.22876367]
 [-0.24625624]
 [-0.24523612]
 [-0.28678118]
 [-0.25771076]
 [-0.30837004]]


After conversion, the train data has a shape:
`[n_samples, n_features (time steps), input_dimension] ---> [811, 11, 1]`
That is, each sample has **11 time steps, each with 1 input dimension**.

Although it applies to a different problem, please see the [this tutorial notebook](https://colab.research.google.com/drive/1zjh0tUPYJYgJJunpLC9fW5uf--O0LKeZ?usp=sharing#scrollTo=ydUsxRzwNl4-) and the example GIF:
<img src="https://github.com/kmkarakaya/ML_tutorials/blob/master/images/conv1d.gif?raw=true" width="500">

Other potentially good papers:
- [Short-Term Traffic Prediction with Vicinity Gaussian Process in the Presence of Missing Data](https://ieeexplore.ieee.org/abstract/document/8547118)
- [A CNN-LSTM-Based Model to Forecast Stock Prices](https://www.hindawi.com/journals/complexity/2020/6622927/)

In [43]:
def create_model():
    # compute number of timesteps and features to pass to model
    n_features = reshaped_x_train.shape[1] # number of features -> 11 in this case
    n_dimension = reshaped_x_train.shape[2] # each feature is represented by 1 number

    model = tf.keras.Sequential(layers=[
        tf.keras.layers.InputLayer(input_shape=(n_features, n_dimension)),
        tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'),
        tf.keras.layers.Dropout(0.4),
        
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'),
        tf.keras.layers.Dropout(0.4),       
               
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=128, activation='relu'),
        tf.keras.layers.Dense(units=n_dimension, activation='linear')
    ])
    return model

cnn_model = create_model()
cnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 11, 32)            128       
_________________________________________________________________
dropout_10 (Dropout)         (None, 11, 32)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 11, 64)            6208      
_________________________________________________________________
dropout_11 (Dropout)         (None, 11, 64)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 704)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               90240     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                

In [44]:
optimisier = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = 'mse'
metric = ['mae']

# compile model
cnn_model.compile(optimizer=optimisier,
                  loss=loss, 
                  metrics=metric)

# fit to training data and record output
history = cnn_model.fit(reshaped_x_train, y_train, epochs=100, verbose=0, validation_split=0.2)

In [45]:
def plot_history(history, metric):
    # convert history dict to df
    data = pd.DataFrame(history.history)
    data['epoch'] = history.epoch
    
    # visualise metric
    train_val_metric = alt.Chart(data).transform_fold(
        [metric, 'val_'+metric],
        as_=['Type', 'Value']
    ).mark_line(clip=True).encode(
        alt.X('epoch:O', title='Epoch'),
        alt.Y('Value:Q',title='Mean Average Error', scale=alt.Scale(domain=(0, 200))),
        alt.Color('Type:N', legend=None)
    ).properties(
    width=300,
    height=300
    )
    
    # visualise loss
    train_val_loss = alt.Chart(data).transform_fold(
        ['loss', 'val_loss'],
        as_=['Type', 'Value']
    ).mark_line(clip=True).encode(
        alt.X('epoch:O', title='Epoch'),
        alt.Y('Value:Q', title='Loss', scale=alt.Scale(domain=(0, 100000))),
        alt.Color('Type:N')
    ).properties(
    width=300,
    height=300
    )
    
    return alt.hconcat(train_val_metric, train_val_loss).resolve_scale(color='independent')

    
plot_history(history, 'mae')

alt.HConcatChart(...)

In [46]:
cnn_y_pred = cnn_model.predict(reshaped_x_test).flatten()
results = cnn_model.evaluate(reshaped_x_test, y_test, verbose=0)
print(f'Test set {loss}: {results[0]:0.2f}')
print(f'Test set {metric}: {results[1]:0.2f}')

Test set mse: 8337.95
Test set ['mae']: 48.50


## 3.4 Model Test Results

In [47]:
def prep_data(predictions, model_names):
    
    # check predictions and mode_names are same length
    if len(predictions) != len(model_names):
        raise ValueError('List of predictions and model names need to be same length')

    # create dataframe of results for each model
    list_of_dfs = []
    for i in range(len(predictions)):
        error = y_test - predictions[i]
        model_name = model_names[i]
        model_data = pd.DataFrame({'x': y_test, 'y': predictions[i],
                                   'error': error, 'model_name': model_name})
        list_of_dfs.append(model_data)

    # concat all individual dfs into one to pass to vis
    data = pd.concat(list_of_dfs, ignore_index=True)
    return data


def visualise_predictions(predictions, model_names):
    '''
    Function to visualise a true vs predicted scatter and histogram of errors
    for each model and prediction in the passed lists.
    
    '''
    data = prep_data(predictions, model_names)

    vis = alt.concat(*(
        # true vs predicted
        alt.Chart(data.loc[data['model_name']==name], title=name+' True vs Predicted'
        ).mark_circle().encode(
            alt.X('x', title='Actual Counts'),
            alt.Y('y', title='Predected Counts'),
            color='model_name:N',
            tooltip=['model_name', 'x', 'y']
        ).properties(
            width=300,
            height=300
        )
        +
        # x=y line
        alt.Chart(pd.DataFrame({'x': [0,3500],
                                'y': [0,3500]})
                        ).mark_line(
            color='grey',
            clip=True
        ).encode(
            alt.X('x'),#, scale=alt.Scale(domain=(0, max_value))),
            alt.Y('y'),#, scale=alt.Scale(domain=(0, max_value)))
        )
        |
        # histogram
        alt.Chart(data.loc[data['model_name']==name], title=name+' Error Histogram'
        ).mark_bar().encode(
            alt.X('error:Q', bin=alt.Bin(maxbins=100, extent=[-500, 500]), title='Prediction Error'),
            alt.Y('count()', title='Count'),
            color='model_name:N',
            tooltip=['model_name','error','count()']
        ).properties(
            width=300,
            height=300
        )
        for name in model_names # [::-1] if want to reverse order
      ), columns=1
    ).properties(
        title = 'Model Test Results',
    )    
    return vis


predictions = [lr_y_pred, rf_y_pred, cnn_y_pred]
model_names = ['LR', 'RF', 'CNN']
visualise_predictions(predictions, model_names)

alt.ConcatChart(...)

In [48]:
def get_mape(y_test, y_pred):
    '''
    Function to replace inf values caused by the division of zero
    with zero so the actual mean can be returned.
    
    '''
    calculation = 100*abs((y_test-y_pred)/y_test)
    for idx, value in enumerate(calculation):
        if np.isinf(value):
            calculation.iloc[idx] = 0
            
    mape = np.mean(calculation) 
    return str(np.round(mape,2))+'%'


def get_test_measures(predictions, model_names):
    '''
    Function to calculate standard statistical measures for each model
    and report them as a DataFrame.
    
    '''
    data = prep_data(predictions, model_names)
    list_of_measures = []
    for name in model_names:
        filtered_data = data.loc[data['model_name']==name]
        y_pred = filtered_data['y']
        measures_dict = {
            'Model Name': name,
            #'Train Time': trainTime,
            'MSE': np.round(np.mean((y_test-y_pred)**2),2),
            'RMSE': np.round(np.sqrt(np.mean((y_test-y_pred)**2)),2),
            'MAE': np.round(np.mean(abs(y_test-y_pred)),2),
            # mean absolute percentage error
            'MAPE': get_mape(y_test, y_pred),
            #'R2': model.score(x, y)
        }
        list_of_measures.append(pd.DataFrame(measures_dict, index=[0]))
        
    measures_df = pd.concat(list_of_measures, ignore_index=True)
    measures_df.set_index('Model Name', inplace=True)
    return measures_df


measures_df = get_test_measures(predictions, model_names)
measures_df

,MSE,RMSE,MAE,MAPE
Model Name,,,,
LR,8362.52,91.45,50.15,21.26%
RF,5137.71,71.68,40.45,18.59%
CNN,8337.95,91.31,48.50,29.08%


# 4. Simulation
In this section we build and run the simulation.

## 4.1 Network
We build the network using OSM data. There are many ways to import a network through OSM data which are discussed on [this page OpenStreetMap](https://sumo.dlr.de/docs/Networks/Import/OpenStreetMap.html). This method required finding the area reference for Swansea (3600087944) and using the [Overpass API Query Form](http://www.overpass-api.de/query_form.html) to download the OSM data.

```
!netconvert --osm-files swansea.osm.xml -o swansea.net.xml --lefthand --geometry.remove --ramps.guess --junctions.join --tls.guess-signals --tls.discard-simple --tls.join --tls.default-type actuated --keep-edges.by-vclass passenger -v
!polyconvert --net-file swansea.net.xml --osm-files swansea.osm.xml --type-file osmPolyconvert.typ.xml -o swansea.poly.xml


Warning: Discarding unknown compound 'usage.main' in type 'railway.rail|usage.main' (first occurence for edge '3993283').
Warning: Discarding unknown compound 'usage.branch' in type 'railway.rail|usage.branch' (first occurence for edge '3993295').
Warning: Discarding unusable type 'waterway.stream' (first occurence for edge '4673837').
Warning: Discarding unknown compound 'cycleway.track' in type 'cycleway.track|highway.secondary' (first occurence for edge '5528275').
Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.service' (first occurence for edge '11359098').
Warning: Discarding unusable type 'highway.construction' (first occurence for edge '12628117').
Warning: Discarding unknown compound 'cycleway.lane' in type 'cycleway.lane|highway.residential' (first occurence for edge '13851148').
Warning: Discarding unusable type 'waterway.canal' (first occurence for edge '23887449').
Warning: Discarding unusable type 'waterway.river' (first occurence for edge '23887457').
    
-----------------------------------------------------
Summary:
 Node type statistics:
  Unregulated junctions       : 0
  Dead-end junctions          : 14
  Priority junctions          : 8033
  Right-before-left junctions : 707
  Traffic light junctions      : 86
 Network boundaries:
  Original boundary  : -4.33,51.54,-3.84,51.78
  Applied offset     : -410710.40,5710965.84
  Converted boundary : 0.00,-25813.54,31168.98,0.00
-----------------------------------------------------
Writing network ... done (2784ms).
Success

Warning: 55 total messages of type: Ambiguity in turnarounds computation at junction '%'.
Warning: 29 total messages of type: Discarding unknown compound '%' in type '%' (first occurence for edge '%').
Warning: 22 total messages of type: Discarding unusable type '%' (first occurence for edge '%').
Warning: 42 total messages of type: Found angle of % degrees at edge '%', segment %.
Warning: 70 total messages of type: Found sharp turn with radius % at the end of edge '%'.
Warning: 81 total messages of type: Found sharp turn with radius % at the start of edge '%'.
Warning: 38 total messages of type: Ignoring restriction relation '%'.
Warning: 105 total messages of type: Intersecting left turns at junction '%' from lane '%' and lane '%' (increase junction radius to avoid this).
Warning: 31 total messages of type: Minor green from edge '%' to edge '%' exceeds %m/s. Maybe a left-turn lane is missing.
Warning: 65 total messages of type: Not joining junctions % (%).
Warning: 21 total messages of type: Reducing junction cluster % (%).
Warning: 249 total messages of type: Speed of % connection '%' reduced by % due to turning radius of % (length=%, angle=%).
Warning: 24 total messages of type: from-edge '%' of restriction relation could not be determined
Warning: 10 total messages of type: to-edge '%' of restriction relation could not be determined
```

In [ ]:
net = sumolib.net.readNet('swansea.net.xml') # read net file

## 4.2 Traffic Demand
It was most appropriate to generate SUMO traffic demand by using randomTrips.py and Calibrators. First, a file containing random routes are created from the python script. Then a calibrator is created for each count point and adds or removes vehicles to the simulation depending on the assigned traffic count that it needs to achieve.

`randomTrips.py` can be called using the following cmd command:

**Note: When using --validate, trip files are generated which results in the simulation running very slowly.** Just use route files.

### 4.2.1 Predict Traffic and Data Prep
In this subsection, we shall predict the traffic using each model. The data shall then be prepared to pass to the randomTrips generator.

In [ ]:
def predict_values(x_data, model, is_cnn=False):
    '''
    Function to predict values from the supplied x_data and model.
    It expects the x_data to be unscaled.
    If the model is the CNN-LSTM model, then reshaping, and a slightly
    different prediction function is called.
    
    '''
    x_data = scale_data(scaler, x_data)
    if is_cnn:
        x_data = reshape_data(x_data)
        predictions = model.predict(x_data).flatten()
        
    else:
        predictions = model.predict(x_data)  
    return predictions


def prediction_df(model_list, x_data):
    '''
    Function to predict values from the supplied x_data 
    for each model in a list and return results in DataFrame.
    
    '''
    prediction_dict = {}
    for idx, model in enumerate(model_list):
        name = model_names[idx]
        if name=='CNN':
            is_cnn=True
        else:
            is_cnn=False
        prediction_dict[name] = predict_values(x_data, model, is_cnn=is_cnn)
        
    return pd.DataFrame(prediction_dict)


model_list = [lr_gscv.best_estimator_, rf_gscv.best_estimator_, cnn_model] # same order as model names
prediction_df = prediction_df(model_list, x)

In [ ]:
def merge_predictions(counts_df, prediction_df):
    '''
    Function to merge the predictions to the counts df and return a new dw.
    Only certain columns specified in counts_df_cols_to_keep are kept.
    The data is then grouped by CP ID and directions of travel, and the mean is computed.
    Finally, all count data is rounded to the nearest integer. 
    
    '''
    # merge data
    counts_df_cols_to_keep = ['count_point_id','direction_of_travel','18']
    merged_df = counts_df[counts_df_cols_to_keep].join(prediction_df)
    
    # group and compute the mean
    merged_df = merged_df.groupby(['count_point_id', 'direction_of_travel']).mean().reset_index()
    
    # round columns
    cols_to_round = ['18','LR','RF','CNN']
    for col in cols_to_round:
        merged_df[col] = np.round(merged_df[col]).astype(int)
    merged_df.rename(columns={'18': 'Actual'}, inplace=True)
    return merged_df
    

sumo_data = merge_predictions(grouped_counts, prediction_df)

In [ ]:
def get_cp_info(dft_counts):
    '''
    Function to extract cp location and road data from the original data.
    
    '''
    cps = dft_counts.groupby('count_point_id').first().reset_index()
    cols_to_keep = ['count_point_id', 'road_name', 'road_type','latitude', 'longitude']
    return cps[cols_to_keep]


def find_closest_edges(coordinates, net, radius=150, edges_to_return=2):
    '''
    Function to find closest n edges to specified coordinates.
    
    '''
    # coordinates must be tuple of latitude then longitude
    x, y = net.convertLonLat2XY(coordinates[1], coordinates[0]) # find x/y positions within network
    edges = net.getNeighboringEdges(x, y, radius) # create list of edges in radius of point
    
    if len(edges) == 0:
        raise ValueError(f'There were no edges found within the {radius} radius of {coordinates}')
    
    # find indices of n closest edges according to distance
    distances = np.array([info[1] for info in edges])
    min_dist_indices = np.argsort(distances)[:edges_to_return]
    closest_edges = np.array(edges)[min_dist_indices] 
    closest_edges = [edge[0] for edge in closest_edges]
    
    return closest_edges


cp_df = get_cp_info(dft_counts)
cp_df['closest_edges'] = [find_closest_edges((lat, long), net, edges_to_return=2) 
                          for lat, long in zip(cp_df['latitude'], cp_df['longitude'])]

In [ ]:
def get_edge_direction(edge):
    '''
    Function to determine which direction a specific edge is going. 
    
    '''
    from_x, from_y, from_z = edge._from._coord
    to_x, to_y, to_z = edge._to._coord
    dx = to_x - from_x
    dy = to_y - from_y
    if abs(dx) > abs(dy): # then edge direction is east or west
        if to_x > from_x: 
            direction = 'E'
        else: 
            direction = 'W'
    if abs(dx) < abs(dy): # then edge direction is north or south
        if to_y > from_y:
            direction = 'N'
        else:
            direction = 'S'
    return direction
    

def match_edges_directions(sumo_data, cp_df):
    '''
    Function to match edges with count point directions.
    In some cases, count points have more than two directions when they are located 
    near junctions/roundabouts; here, the edge is set to NaN, and is dropped for simplicity.
    Note: only two edges are retrieved in the find_closest_edges function. 
    
    '''
    start = time.time()
    
    edge_col_list = []
    for cp_id in cp_df['count_point_id']:
        cp_actual_dirs = [actual_dir for actual_dir in
                          sumo_data.loc[sumo_data['count_point_id']==cp_id]['direction_of_travel']]
        edges = cp_df.loc[cp_df['count_point_id']==cp_id]['closest_edges'].iloc[0]
        edge_dirs = [get_edge_direction(edge) for edge in edges]
        for direction in cp_actual_dirs:
            if direction in edge_dirs:
                idx = edge_dirs.index(direction)
                cp_actual_dirs[cp_actual_dirs.index(direction)] = edges[idx]
            else: 
                cp_actual_dirs[cp_actual_dirs.index(direction)] = np.nan
        edge_col_list = edge_col_list + cp_actual_dirs
    sumo_data['edge'] = edge_col_list
    sumo_data.dropna(axis=0, inplace=True)
    sumo_data.reset_index(drop=True, inplace=True)
    
    print(f'Matched edges and directions in {np.round(time.time()-start, 2)} seconds')
    return sumo_data


sumo_data = match_edges_directions(sumo_data, cp_df)

In [ ]:
def impute_edge_speeds(sumo_data):
    '''
    Function to extract the average speed from each edge and impute as new column. 
    
    '''
    edge_speeds = [edge._speed for edge in sumo_data['edge']]
    
    MPS_TO_KPH = 3.6 # MPS to KPH    
    # take 5 mph off max road speed to better represent avg speed of vehicles
    # sumo_data['avg_speed'] = [speed - (5/MPH_TO_MPS) for speed in edge_speeds]
    
    sumo_data['avg_speed'] = edge_speeds
    return sumo_data
    
sumo_data = impute_edge_speeds(sumo_data)

In [ ]:
def plot_cps(cp_df):
    '''
    Function to plot cps on a folium interactive map. 
    
    '''
    swansea_coords = [51.6195955, -3.9459248]
    m = folium.Map(location=swansea_coords, zoom_start=11)   
    for index, row in cp_df.iterrows():
        folium.Marker((row.latitude, row.longitude), popup = row.count_point_id,
                      icon=folium.Icon(color='black')).add_to(m)
    return m


plot_cps(cp_df)

In [ ]:
sumo_data

### 4.2.2 Generate Traffic Demand
In this section we create the randomTrips and calibrator file. 

In [ ]:
SWANSEA_NET_FILENAME = 'swansea.net.xml'
TRAFFIC_TRIPS_FILENAME = 'traffic_trips.xml'
TRAFFIC_ROUTES_FILENAME = 'traffic_routes.xml'

```
!cd C:\Users\ollir\OneDrive\Documents\University\Data Science MSc\Dissertation\To-Fly-or-Not-to-Fly\simulation
!randomTrips.py -n {SWANSEA_NET_FILENAME} -o {TRAFFIC_TRIPS_FILENAME} --route-file {TRAFFIC_ROUTES_FILENAME} -e 3600 -p 0.25

calling C:\Program Files (x86)\Eclipse\Sumo\bin\duarouter -n swansea.net.xml -r traffic_trips.xml --ignore-errors --begin 0 --end 3600 --no-step-log --no-warnings -o traffic_routes.xml
Success.
```

In [ ]:
CALIBRATOR_FILENAME = "calibrator.xml"
CALIBRATOR_OUTPUT_FILENAME = 'calibrator_output.xml'
ROUTE_PROBE_OUTPUT_FILENAME = 'route_probe_output.xml'
EDGE_OUTPUT_FILENAME = 'edge_output.xml'


def generate_calibrator_file(model_name, sumo_data=sumo_data):
    '''
    Function to generate the SUMO detector file from the sumo_data df.
    '''
    FREQ = 1 # the aggregation interval in which to calibrate the flows. default is step-length
    
    with open(CALIBRATOR_FILENAME, 'w') as outf:
        outf.write("<additional>\n")
        
        # route probes
        for index, row in sumo_data.iterrows():
            outf.write(f"    <routeProbe id='{row.count_point_id+'_'+row.direction_of_travel+'_probe'}' edge='{row.edge._id}' freq='{FREQ}' file='{ROUTE_PROBE_OUTPUT_FILENAME}'/>\n".replace("'",'"'))
        
        # calibrators
        for index, row in sumo_data.iterrows():
            
            outf.write(f"    <calibrator id='{row.count_point_id+'_'+row.direction_of_travel}' edge='{row.edge._id}' pos='{row.edge._lanes[0]._length/2}' freq='{FREQ}' routeProbe='{row.count_point_id+'_'+row.direction_of_travel+'_probe'}' output='{CALIBRATOR_OUTPUT_FILENAME}'>\n".replace("'",'"'))
            outf.write(f"        <route id='{'fallback_'+row.count_point_id+'_'+row.direction_of_travel}' edges='{row.edge._id}'/>\n".replace("'",'"'))
            outf.write(f"        <flow  begin='0' end='3600' route='{'fallback_'+row.count_point_id+'_'+row.direction_of_travel}' vehsPerHour='{row[model_name]}' speed='{row.avg_speed}' vType='DEFAULT_VEHTYPE'/>\n".replace("'",'"'))
            outf.write("    </calibrator>\n")
        
        # generate edge-based data output
        outf.write(f"    <edgeData id='measure_1' file='{EDGE_OUTPUT_FILENAME}'/>\n".replace("'",'"'))      
        
        outf.write("</additional>")


note: [position of calibrator is actually ignored at the moment](https://github.com/eclipse/sumo/issues/1331).

`freq` attribute of the calibrator may need to be increased as there is a trade of between small and large time intervals in which to calibrate the flow.

"While this can be done with dfrouter as well, the method described here is more robust for highly meshed networks as found in cities"

"For the calibrator to be able to function before the first vehicle, it needs a fall back route which just needs to consist of a single edge (i.e. the edge on which the calibrator is placed)."

"However, the realism of traffic flow behind (or between) calibrators depends on the fit between random routes and real-world routes."

### 4.2.3 Generate EV File
The file describing the EV vehicle type and route to travel is generated in this subsection.

In [ ]:
EV_FILENAME = 'emergency_vehicle.xml'


def generate_ev_file():
    '''
    Function to generate the SUMO file describing the EV vehicle type
    
    '''
    with open(EV_FILENAME, 'w') as outf:
        outf.write("<routes>\n")
        
        # vType
        outf.write("    <vType id='ev' length='5' maxSpeed='67' accel='4' sigma='0.2' speedFactor='1.5' guiShape='emergency' vClass='emergency'>\n".replace("'",'"'))
        outf.write("        <param key='has.tripinfo.device' value='true'/>\n".replace("'",'"'))
        outf.write("        <param key='has.bluelight.device' value='true'/>\n".replace("'",'"'))
        outf.write("        <param key='has.fcd.device' value='true'/>\n".replace("'",'"'))
        outf.write("    </vType>\n")
        
        # this is from Morriston Hospital to Singleton Hospital
        outf.write("    <trip id='ev_trip' type='ev' depart='600' from='-41432539#2' to='836332232#0'/>\n".replace("'",'"'))
        outf.write("    <!-- this is from Morriston Hospital -> Singleton Hospital -->\n")
        
        outf.write("</routes>")


## 4.3 Run the Simulation
Once all traffic demand files have been generated, the simulation can be run

In [ ]:
SUMO_CONFIG_FILENAME = 'swansea.sumocfg'
SWANSEA_POLY_FILENAME = 'swansea.poly.xml'
TRIPINFO_OUTPUT_FILENAME = 'tripinfo_output.xml'
FCD_OUTPUT_FILENAME = 'fcd_output.xml'
STATS_OUTPUT_FILENAME = 'statistic_output.xml'
SIMULATION_VIEW_FILENAME = 'simulation_view.xml'


def generate_sim_view():
    '''
    Function to generate the GUI view settings file.
    This file was really only necessary in testing using the GUI application.
    
    '''
    with open(SIMULATION_VIEW_FILENAME, 'w') as outf:
        outf.write("<viewsettings>\n")
        outf.write("    <scheme name='real world'/>\n".replace("'",'"'))
        outf.write("    <delay value='0'/>\n".replace("'",'"'))
        outf.write("</viewsettings>")


def generate_sumo_config():
    '''
    Function to generate the sumo config file for each simulation.
    
    '''
    with open(SUMO_CONFIG_FILENAME, 'w') as outf:
        outf.write("<configuration>\n")
        
        # input
        outf.write("    <input>\n")
        outf.write(f"        <net-file value='{SWANSEA_NET_FILENAME}'/>\n".replace("'",'"'))
        outf.write(f"        <route-files value='{EV_FILENAME},{TRAFFIC_ROUTES_FILENAME}'/>\n".replace("'",'"'))
        outf.write(f"        <additional-files value='{SWANSEA_POLY_FILENAME},{CALIBRATOR_FILENAME}'/>\n".replace("'",'"'))
        outf.write("        <begin value='0'/>\n".replace("'",'"'))
        outf.write("        <end value='3600'/>\n".replace("'",'"'))
        outf.write("        <seed value='202'/>\n".replace("'",'"'))
        outf.write("    </input>\n")
        
        # lane changing model
        #outf.write("    <lane_changing>\n")
        #outf.write("        <lateral-resolution value='3.2'/>\n".replace("'",'"'))
        #outf.write("    </lane_changing>\n")
        
        # processing
        outf.write("    <processing>\n")
        outf.write("        <ignore-route-errors value='true'/>\n".replace("'",'"'))
        outf.write("    </processing>\n")
        
        
        # devices
        outf.write("    <devices>\n")
        outf.write("        <device.bluelight.explicit value='ev_trip'/>\n".replace("'",'"'))
        outf.write("        <device.fcd.explicit value='ev_trip'/>\n".replace("'",'"'))
        outf.write("        <device.rerouting.explicit value='ev_trip'/>\n".replace("'",'"'))
        outf.write("        <device.rerouting.period value='100'/>\n".replace("'",'"'))
        outf.write("    </devices>\n")
        
        # report
        outf.write("    <report>\n")
        outf.write("        <verbose value='true'/>\n".replace("'",'"'))
        outf.write("        <duration-log.statistics value='true'/>\n".replace("'",'"'))
        outf.write("        <no-step-log value='true'/>\n".replace("'",'"'))
        outf.write(f"        <tripinfo-output value='{TRIPINFO_OUTPUT_FILENAME}'/>\n".replace("'",'"'))
        outf.write(f"        <fcd-output value='{FCD_OUTPUT_FILENAME}'/>\n".replace("'",'"'))
        outf.write(f"        <statistic-output value='{STATS_OUTPUT_FILENAME}'/>\n".replace("'",'"'))
        outf.write("    </report>\n") 
        
        # gui settings
        outf.write("    <gui_only>\n")
        outf.write(f"        <gui-settings-file value='{SIMULATION_VIEW_FILENAME}'/>\n".replace("'",'"'))
        outf.write("    </gui_only>\n")        
        
        outf.write("</configuration>")
    

Need some way to handle the output files. This involves converting them from XML files to Pandas DataFrames. This may have to be done via conversion to CSV first, but Pandas do have a `read_xml` function that can read some flatter XML documents according to [the documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_xml.html).

In [ ]:
tripinfo_output_df_list = []
ev_output_df_list = []
fcd_output_df_list = []
stats_output_df_list = []
edge_output_df_list = []
calibrator_output_df_list = []


def import_tripinfo(model_name):
    '''Function to import tripinfo output'''
    tripinfo_output = pd.read_xml(TRIPINFO_OUTPUT_FILENAME)
    cols = ['id','depart','arrival','duration','routeLength','waitingTime', 'timeLoss','vaporized']
    tripinfo_output = tripinfo_output[cols]
    tripinfo_output['model_name'] = model_name
    
    # extract the ev trip data
    ev_output = tripinfo_output.loc[tripinfo_output['id']=='ev_trip']
    ev_output_df_list.append(ev_output)
    
    # drop ev trip data
    tripinfo_output = tripinfo_output.drop(ev_output.index[0]).reset_index(drop=True)
    
    tripinfo_output_df_list.append(tripinfo_output)
    
    
def import_fcd(model_name):
    '''Function to import fcd output'''
    fcd_output = pd.read_xml(FCD_OUTPUT_FILENAME, xpath=".//vehicle")
    fcd_output['model_name'] = model_name
    fcd_output_df_list.append(fcd_output)
        

def import_stats(model_name):
    '''Function to import stats output'''
    stats_output = pd.read_xml(STATS_OUTPUT_FILENAME)
    
    # extract numeric values into a series
    stats_dict = {}
    for col in stats_output.columns:
        stats_dict[col] = np.array(stats_output[col].dropna())[0]
    
    stats_dict['model_name'] = model_name    
    stats_output_df_list.append(pd.Series(stats_dict))
    
    
def import_edge(model_name):
    '''Function to import edge output'''
    edge_output = pd.read_xml(EDGE_OUTPUT_FILENAME, xpath=".//edge")
    cols = ['id','sampledSeconds','traveltime','density','occupancy',
            'waitingTime', 'timeLoss','speed', 'vaporized']
    edge_output = edge_output[cols]
    edge_output['model_name'] = model_name
    edge_output_df_list.append(edge_output)
    
    
def import_calibrator(model_name):
    '''Function to import calibrator output'''
    calibrator_output = pd.read_xml(CALIBRATOR_OUTPUT_FILENAME)
    cols = ['id','nVehContrib','removed','inserted','cleared','flow',
            'aspiredFlow', 'speed', 'aspiredSpeed']
    calibrator_output = calibrator_output[cols]
    calibrator_output['model_name'] = model_name
    calibrator_output_df_list.append(calibrator_output)


In [ ]:
def run_simulations(model_names):
    '''
    Function to run the simulations for each model.
    
    '''
    for model_name in model_names:
        start = time.time()
        
        # generate xml files
        generate_calibrator_file(model_name=model_name)
        generate_ev_file()
        generate_sim_view()
        generate_sumo_config()  
        print('Generated all xml files')
        
        # run simulation
        print('Starting simulation...')
        !cd C:\Users\ollir\OneDrive\Documents\University\Data Science MSc\Dissertation\To-Fly-or-Not-to-Fly\simulation
        !sumo -c {SUMO_CONFIG_FILENAME}
        print(f'Generated files and ran simulation for {model_name} model in {np.round(time.time()-start, 2)} seconds')
        
        # convert output files to DataFrame and add to list
        import_tripinfo(model_name)
        import_fcd(model_name)
        import_stats(model_name)
        import_edge(model_name)
        import_calibrator(model_name)


model_names = ['Actual','LR', 'RF', 'CNN']
run_simulations(model_names)        

In [ ]:
%store tripinfo_output_df_list
%store ev_output_df_list
%store fcd_output_df_list
%store stats_output_df_list
%store edge_output_df_list
%store calibrator_output_df_list

In [ ]:
# %store -r tripinfo_output_df_list
# %store -r ev_output_df_list
# %store -r fcd_output_df_list
# %store -r stats_output_df_list
# %store -r edge_output_df_list
# %store -r calibrator_output_df_list

In [ ]:
# create single dataframes from all simulation runs
tripinfo_output = pd.concat(tripinfo_output_df_list)
ev_output = pd.concat(ev_output_df_list)
fcd_output = pd.concat(fcd_output_df_list)
stats_output = pd.concat(stats_output_df_list, axis=1)
edge_output = pd.concat(edge_output_df_list)
calibrator_output = pd.concat(calibrator_output_df_list) 

# 5. Visualise Simulation Results
In this section we visualise the simulation outputs.

## 5.1 Trip Info
First, we visualise the `tripinfo_output` of the simulations.

In [ ]:
tripinfo_output

In [ ]:
tripinfo_output.groupby(['model_name']).median()

In [ ]:
# disable max rows warning
alt.data_transformers.disable_max_rows()

def depart_arrive():
    '''
    Visual to show how many vehicles departing/arriving as the simulation progresses.
    
    '''
    depart_arrive = alt.Chart(tripinfo_output).transform_fold(
        ['depart','arrival'],
        as_=['Action','Time']
    ).mark_line(
        opacity=1
    ).encode(
        alt.X('Time:O', bin=alt.Bin(maxbins=50), title='Time of Departure in Simulation'),
        alt.Y('count()', stack=None),
        alt.Color('Action:N')
    ).properties(
        width=200,
        height=200
    ).facet(
        column='model_name:N',
        title='Departure vs Arrival Counts'
    )

    return depart_arrive

depart_arrive()

In [ ]:
def boxes():
    '''
    Visual to show box plots of trip durations, routeLength, waitingTime, and timeLoss
    for each simulation. 
    Note: these return the MEDIAN (more robust to outliers)
    
    '''
        
    features=['duration','routeLength','waitingTime','timeLoss']
    
    boxes = alt.concat(*(
        alt.Chart(tripinfo_output, title=feature.title()).mark_boxplot(
            outliers=False
        ).encode(
            alt.X('model_name:N', title='Model Name'),
            alt.Y(feature+':Q', title=feature.capitalize()),
            alt.Color('model_name:N', legend=None)
        ).properties(
            width=125,
            height=125
        )
        for feature in features
      ), columns=4
    ).properties(
            title = 'Boxplots of Traffic Route Data'
    )  
    
    return boxes


boxes()

In [ ]:
def vaporised():
    '''
    Visual to show how many vehicles were vaporised and why. 
    
    '''
    vaporised = alt.Chart(tripinfo_output).transform_filter(
        alt.FieldOneOfPredicate(field='vaporized', oneOf=['calibrator', 'teleport'])
    ).mark_line(
        opacity=1
    ).encode(
        alt.X('depart:O', bin=alt.Bin(maxbins=50), title='Time of Departure in Simulation'),
        alt.Y('count():Q'),
        alt.Color('vaporized'),
        tooltip=['count()']
    ).properties(
        width=200,
        height=200
    ).facet(
        column='model_name:N',
        title='Vaporised Vehicles'
    )

    return vaporised


vaporised()

## 5.2 Emergency Vehicles
Visualise the emergency vehicles and their tripinfo and FCD output.

In [ ]:
ev_output

In [ ]:
def ev_tripinfo():
    '''Visual of the EV route information'''

    tripinfo = alt.Chart(ev_output).mark_bar().encode(
        alt.Y('model_name:N', title=None),
        alt.X(alt.repeat('row'), type='quantitative'),
        color='model_name:N',
        tooltip=[
            alt.Tooltip('model_name:N', title='Model Name'),
            alt.Tooltip(alt.repeat('row'), type='quantitative')]
    ).repeat(
        row=['routeLength','duration','waitingTime','timeLoss']
    ).properties(
        title='EV Trip Information per Simulation'
    )
    
    return tripinfo


ev_tripinfo()

In [ ]:
def convert_xy_to_latlon(df):
    '''
    Function to convert SUMO x and y coordinates to longitude
    and latitude.
    
    '''
    points_lonlat = [net.convertXY2LonLat(x,y) for x,y in zip(df['x'], df['y'])]
    df['latitude'] = [lat for lon, lat in points_lonlat]
    df['longitude'] = [lon for lon, lat in points_lonlat]
    return df
    

fcd_output = convert_xy_to_latlon(fcd_output)

In [ ]:
def plot_ev_routes_folium(df):
    '''
    Function to plot EV routes from the FCD output df.
    
    '''
    # initiate map
    swansea_coords = [51.6195955, -3.9459248]
    m = folium.Map(location=swansea_coords, zoom_start=11)
    
    
    # iterate over models
    # route_colours = ['#808080','#69BE28','#3DB7E4',] # grey, green, blue
    route_colours = ['green','red','blue','orange']
    for idx, model in enumerate(model_names):
        filtered_df = df.loc[df['model_name']==model]
        route = [(lon, lat) for lon, lat in zip(filtered_df['latitude'],filtered_df['longitude'])]
        folium.PolyLine(route, tooltip=model, color=route_colours[idx], opacity=1).add_to(m)
        
    
    # origin marker
    origin = (filtered_df.iloc[0]['latitude'],filtered_df.iloc[0]['longitude'])
    folium.Marker(origin, popup='Morriston Hospital', 
                  icon=folium.Icon(color='green',
                                   icon='h-square', prefix='fa')).add_to(m)
    
    # destination marker
    destination = (filtered_df.iloc[-1]['latitude'],filtered_df.iloc[-1]['longitude'])
    folium.Marker(destination, popup='Singleton Hospital', 
                  icon=folium.Icon(color='red',
                                   icon='h-square', prefix='fa')).add_to(m)
    
    return m


plot_ev_routes_folium(fcd_output)

In [ ]:
def merge_ev_fcd(ev_df, fcd_df):
    '''
    Function to take the route of each EV from fcd_output, 
    and merge it into ev_output as a polyline geometry.
    
    '''
    routes_coords_list = []
    for model in model_names:
        filtered_fcd = fcd_df.loc[fcd_df['model_name']==model]
        route_coords = [(lon, lat) for lon, lat in zip(filtered_fcd['longitude'], filtered_fcd['latitude'])]
        routes_coords_list.append(route_coords)
        
    ev_df['geometry'] = [LineString(route) for route in routes_coords_list]
    merged_df = gpd.GeoDataFrame(ev_df)    
    return merged_df


ev_output = merge_ev_fcd(ev_output, fcd_output)

In [ ]:
def get_edge_geometries(df, edge_col_name, using='edge_id'):
    '''
    Function to add a column containing the edge geometry in a 
    GeoPandas geometry form. These can be obtained from Edge object, or Edge IDs.
    This is done by looking up the edge shape from the SUMO net.
    Then the (x,y) coordinates are converted to (lon,lat) and finally,
    converted to GeoPandas LineStrings.  
    
    '''
    lines_lonlat_list = []
    for edge in df[edge_col_name]:
        if using == 'edge_id':
            lines_xy = net.getEdge(edge).getShape() # edge_id
        elif using == 'edge_object':
            lines_xy = edge.getShape() # edge_object
        lines_lonlat_list.append([net.convertXY2LonLat(x,y) for x,y in lines_xy])
    df['geometry'] = [LineString(line) for line in lines_lonlat_list]
    df_gdf = gpd.GeoDataFrame(df)        
    return df_gdf
        

def get_edge_df():
    edge_objects = net.getEdges()
    edge_df = pd.DataFrame({'edges': edge_objects})
    edge_df = get_edge_geometries(edge_df, edge_col_name='edges', using='edge_object')
    return edge_df


edge_df = get_edge_df()

In [ ]:
fcd_output

In [ ]:
def plot_ev_routes_altair():
    '''
    Function to plot EV routes on a map in Altair.
    
    '''
    roads = alt.Chart(edge_df['geometry']).mark_geoshape(
        filled=False,
        strokeWidth=1,
        color='lightgrey'
    ).properties(
        width=600,
        height=400
    )

    ev_routes = alt.Chart(ev_output).mark_geoshape(
        filled=False,
        strokeWidth=2
    ).encode(
        alt.Color('model_name:N'),
        tooltip=[alt.Tooltip('model_name:N', title='Model Name'),
                 alt.Tooltip('duration:Q', title='Duration (s)'),
                 alt.Tooltip('waitingTime:Q', title='Waiting Time (s)'),
                 alt.Tooltip('timeLoss:Q', title='Time Loss (s)')]
    )
    
    speed_pos = alt.Chart(fcd_output).mark_circle().encode(
        alt.Longitude('longitude:Q', title='Longitude'),
        alt.Latitude('latitude:Q', title='Latitude'),
        color='speed:Q',
        tooltip=['speed:Q']
    ).properties(
        width=200,
        height=200
    ).facet(
        column='model_name:N'
    )
    
    
    return (roads + ev_routes) & speed_pos & tripinfo
    

plot_ev_routes_altair()

In [ ]:
cp_df

In [ ]:
def plot_cps_altair():
    '''
    Map of CPs around Swansea
    
    '''
    roads = alt.Chart(edge_df['geometry']).mark_geoshape(
        filled=False,
        strokeWidth=1,
        color='lightgrey'
    ).properties(
        width=600,
        height=400
    )
    
    cps = alt.Chart(cp_df.drop(columns=['closest_edges'])).mark_circle().encode(
        alt.Longitude('longitude:Q', title='Longitude'),
        alt.Latitude('latitude:Q', title='Latitude'),
        tooltip=[alt.Tooltip('count_point_id:N', title='Count Point ID')]
    )
    
    return roads + cps


plot_cps_altair()

## 5.3 Simulation Statistics
Here we visualise simulation statistics. To be honest, this data may just be better presented in a table. 

In [ ]:
stats_output

## 5.4 Edge Data
Here the edge output data is visualised to give some location context.

In [ ]:
def derive_further_values(df):
    '''
    Function to derive further values from the edge output data
    as suggested on the SUMO wiki page:
    https://sumo.dlr.de/docs/Simulation/Output/Lane-_or_Edge-based_Traffic_Measures.html
    
    '''
    df['avg_num_veh'] = df['sampledSeconds']/3600 # every hour
    df['avg_vol'] = df['speed']*3.6*df['density']    
    df.fillna(0, inplace=True)
    return df


edge_output = derive_further_values(edge_output)

In [ ]:
edge_output = get_edge_geometries(edge_output, edge_col_name='id', using='edge_id')
edge_output

In [ ]:
edge_output.groupby(['model_name']).mean()

In [ ]:
alt.Chart(edge_output.drop(columns['geometry'])).mark_boxplot(
    outliers=False
).encode(
    alt.Y('model_name:N', title='Model Name'),
    alt.X('speed:Q', title='Speed (m/s)'),
    alt.Color('model_name:N')
)

In [ ]:
edge_output.loc[edge_output['model_name']=='CNN'].hist(column='speed')

In [ ]:
def edge_map():
    '''Visual of the edge map, coloured by some feature.'''
      
    edge_map = alt.concat(*(
        alt.Chart(edge_output.loc[edge_output['model_name']==name], title=name).mark_geoshape(
            filled=False,
            strokeWidth=1
        ).encode(
            alt.Color('speed:Q', scale=alt.Scale(scheme='yellowgreenblue')),#, bin=alt.Bin(maxbins=6)),
            tooltip=['id','speed']
        ).properties(
            width=600,
            height=400
        )
        for name in model_names
      ), columns=1
    ).properties(
        title = 'Mean Edge Speed of Simulations'
    )  

    return edge_map


## 5.5 Calibrator Data
In this section we visualise the calibrator output data for each simulation.

In [ ]:
def add_edges_to_calibrator_file():
    '''
    Function to add edges to the calibrator output file.
    
    '''
    calibrator_edges = pd.read_xml(CALIBRATOR_FILENAME, xpath=".//calibrator")
    return pd.merge(calibrator_output, calibrator_edges[['id','edge']], on = 'id', how = 'inner')


calibrator_output = add_edges_to_calibrator_file()
calibrator_output = get_edge_geometries(calibrator_output, edge_col_name='edge', using='edge_id')

In [ ]:
calibrator_output

## 5.6 Simulation Results DataFrame
Just a table of simulation numerical results

# To Do List

Just some things to revist and improve:

#### Simulation wise
- Double check that the removal of certain vehicle classes has been set correctly for the calibrators. The simulation will need to be re-run to do this. 
    - [This forum post](https://www.eclipse.org/lists/sumo-user/msg06014.html) was useful, as well as [the Calibrator page](https://sumo.dlr.de/docs/Simulation/Calibrator.html) on this SUMO wiki which stated `the normal behavior is to replace the type of the passing vehicles with the type set in the flow element.`
- Try and get the sublane model to work. This is set by putting `<lateral-resolution value="3.2"/>` in the `sumocfg` file. Take a look at the [Sublane Model wiki page](https://sumo.dlr.de/docs/Simulation/SublaneModel.html) for more details about specific parameters that can be set.
    - Can look at [this vType deafault parameter page](https://sumo.dlr.de/docs/Vehicle_Type_Parameter_Defaults.html) for vehicle widths.
- Also worth exploring how to disregard right-of-way and traffic lights. As per the [Emergency Vehicle simulation page](https://sumo.dlr.de/docs/Simulation/Emergency.html) this may have to be done through `TraCI speed mode command` but unsure if that is a feasible solution for my goal.

#### Writing wise
- During search for answers regarding sublane model, came across multiple papers by `Laura Bieker-Walz` from the German Aerospace Center that focused on simulating EVs in SUMO. [The obrusnik paper](https://wiki.control.fel.cvut.cz/mediawiki/images/0/00/Dp_2019_obrusnik_vit.pdf) mentioned a number of sources in the `EV modelling` section. Two papers I came across can be found [here](https://elib.dlr.de/118034/1/SUMO2017_bieker_emergency_vehicle_final.pdf) and [here](file:///C:/Users/ollir/Downloads/Analysis_of_the_traffic_behavior_of_emergency_vehicles_in_a_microscopic_traffic_simulation.pdf).

#### Visualisation wise
- All of it. Refer to the other jupyter notebook for a starting point
